# Analyse des données de valeurs foncières (2017-2020)

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio; pio.renderers.default='notebook'
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Récupération des .csv

In [2]:
# Import des .csv via une list comprehension
listecsv = ['dvf_2017','dvf_2018','dvf_2019','dvf_2020']
file_name = '{}.csv'
df_list = []
df = pd.concat([pd.read_csv(file_name.format(i)) for i in listecsv])

In [ ]:
df.shape

In [ ]:
df.head()

### Feature engineering

In [3]:
#Suppression des colonnes suivantes
del df['Code commune']
del df['Prefixe de section']
del df['Section']
del df['Code voie']
del df['Voie']
del df['No voie']
del df['Type de voie']

In [4]:
# Renommage de la colonne "Unnamed 0" en "numero"
df.rename(columns={'Unnamed: 0': 'numero'}, inplace=True)

In [5]:
# Ajout d'un zéro pour les départements concernés
df["Code departement"] = df["Code departement"].apply(lambda x: '0' + str(x) if len(str(x)) < 2 else str(x))

In [6]:
# Remplacement des NaN de Type local par "Autres"
df['Type local'] = df['Type local'].replace(np.nan, "Autres")

In [7]:
df['Date mutation'] = pd.to_datetime(df['Date mutation'], errors='coerce')

In [8]:
# Création de la colonne "Année"
df['Annee'] = pd.DatetimeIndex(df['Date mutation']).year
df['Mois'] = pd.DatetimeIndex(df['Date mutation']).month
df['Jour'] = pd.DatetimeIndex(df['Date mutation']).day

In [21]:
# Subset par année 
df_2017 = df[df['Annee'] == 2017]
df_2018 = df[df['Annee'] == 2018]
df_2019 = df[df['Annee'] == 2019]
df_2020 = df[df['Annee'] == 2020]

In [9]:
df.head()

,numero,No disposition,Date mutation,Nature mutation,Valeur fonciere,Code postal,Commune,Code departement,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,Annee,Mois,Jour
0,0,1,2017-02-01,Vente,27000.0,01000,BOURG-EN-BRESSE,01,Appartement,37.0,2.0,NaN,NaN,2017,2,1
1,1,1,2017-05-01,Vente,115000.0,01960,PERONNAS,01,Autres,NaN,NaN,AB,788.0,2017,5,1
2,2,1,2017-06-01,Vente,1.0,01380,SAINT-CYR-SUR-MENTHON,01,Autres,NaN,NaN,P,42.0,2017,6,1
3,3,1,2017-06-01,Vente,1.0,01380,SAINT-CYR-SUR-MENTHON,01,Autres,NaN,NaN,P,77.0,2017,6,1
4,4,1,2017-06-01,Vente,1.0,01380,SAINT-CYR-SUR-MENTHON,01,Autres,NaN,NaN,P,94.0,2017,6,1


### Exploratory Data Analysis - EDA

#### Valeurs foncières en fonction de l'année : 

In [ ]:
fig = px.sunburst(df, path=['Annee'], values = 'Valeur fonciere',
                  color_discrete_sequence=px.colors.diverging.Spectral,
                  title = 'Part relative de la valeur foncière totale en fonction des années')
fig.update_traces(textinfo="label+percent entry")
fig.show()

#### Type de mutations par année :

In [ ]:
fig = px.sunburst(df, path=['Annee','Nature mutation'], values = 'Valeur fonciere',
                  color_discrete_sequence=px.colors.diverging.Spectral,
                  title = 'Part relative du type de mutations en fonction des années')
fig.update_traces(textinfo="label+percent entry")
fig.show()

#### Nombre de mutation par département :

In [ ]:
fig = px.histogram(df, x="Code departement",y="numero",histfunc='count')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

#### Type de local par année :

In [ ]:
fig = px.histogram(df, x='Type local', y = 'numero', color = 'Type local',
                   histfunc = 'count', facet_col='Annee', facet_col_wrap=2)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

#### Type de mutations par année dans le Rhône (69) :

In [10]:
df_rhone = df[df['Code departement'] == '69']

In [ ]:
df_rhone.head()

In [ ]:
fig = px.sunburst(df_rhone, path=['Annee','Commune','Nature mutation'], values = 'Valeur fonciere',
                  color_discrete_sequence=px.colors.diverging.Spectral)
fig.update_traces(textinfo="label+percent entry")
fig.show()

#### Evolution du nombre de mutations sur Lyon (2017-2020) : 

In [11]:
df_lyon = df_rhone[df_rhone['Commune'].str.contains("^LYON")]

In [25]:
df_grouped = df_lyon.groupby(['Commune','Annee'],as_index=False).count()

In [ ]:
px_plot = px.line(df_grouped, x='Annee', y='numero', color = 'Commune',
                  color_discrete_sequence=px.colors.qualitative.Bold,markers=True)
px_plot.update_traces(marker={'size': 12})
px_plot

### Analyse time series

#### Evolution du nombre de mutations à lyon :

In [ ]:
fig = px.histogram(df_lyon, x="Date mutation",y="numero",histfunc='count', color = 'Commune',
                   facet_col='Commune', facet_col_wrap=3)
fig.show()

#### Recherche de saisonnalité : 

In [ ]:
fig = px.histogram(df_lyon, x="Mois",y="numero",histfunc='count', color = 'Commune',
                   facet_col='Commune', facet_col_wrap=3)
fig.show()

In [ ]:
fig = px.histogram(df_lyon, x="Jour",y="numero",histfunc='count', color = 'Commune',
                   facet_col='Commune', facet_col_wrap=3)
fig.show()